### Redis Data Types
Redis has 5 primary data types:
- STRING: used to store strings and numbers
- LIST: linked list of STRING
- SET: unordered collection of unique STRING
- HASH: key value pair
- ZSET: sorted SET

All data stored in Redis is in key-value form. Value can be either of the 5 types described above

### STRING
The String data type is used to store String, Integer and Floating point numbers. The basic operations associated with STRING are `GET`,`SET` and `DEL`.

In [ ]:
const redis = require('/usr/local/lib/node_modules/redis')
const client = redis.createClient()

In [3]:
function print(err, val){
    if(!err){
        console.log(val)
    } else {
        console.log(err)
    }
}

// Creating a string value
client.set('mykey', 'Hello', print)
client.get('mykey', print)

true

OK
Hello


Since the same STRING type is used to represent Integers and Floating type data, we have number specific commands. Note that a nonexisting key or blank string is equivalent to 0. The cli commands `INCR`, `DECR`, `INCRBY`, `DECRBY`

In [4]:
client.set('mynum', '50')
client.incr('mynum', print)
client.incrby('mynum', 5, print)

true

51
56


In [5]:
// Key doesn't exist
client.decr('unknown', print)

true

-1


In [6]:
// Key containing string literal
client.decrby('mykey', 10, print)

true

{ ReplyError: ERR value is not an integer or out of range
    at parseError (/usr/local/lib/node_modules/redis/node_modules/redis-parser/lib/parser.js:179:12)
    at parseType (/usr/local/lib/node_modules/redis/node_modules/redis-parser/lib/parser.js:302:14) command: 'DECRBY', args: [ 'mykey', 10 ], code: 'ERR' }


A few traditional string operations are also supported. `APPEND`, `GETRANGE` (equivalent to substring), `SETRANGE` and more bit related functions are available.

In [8]:
client.set('greeting', 'Hello')
client.append('greeting', 'World', print)

client.getrange('greeting', 0, 4, print) // both inclusive, the string length is printed
client.get('greeting', print)

client.setrange('greeting', 5, 'Earth', print) // replacing World with Earth, string length is printed
client.get('greeting', print)

true

10
Hello
HelloWorld
10
HelloEarth


### LIST
Ordered sequence of STRING. The typical operations are what we expect from lists `LPUSH/RPUSH` (add to beginning or end), `LINDEX` (get item at index), `LRANGE` (get range of items), `LPOP/RPOP` (pop element from either end), `LLEN` (get length of list)

In [11]:
client.rpush('cities', 'new york', 'london', 'paris', print)
client.lindex('cities', 0, print)

true

3
new york


In [12]:
client.lpop('cities', print)
client.llen('cities', print)

true

new york
2


In [13]:
// Get the entire list
client.llen('cities', (err, res)=>{
    client.lrange('cities', 0, res-1, (err, res)=>{
        console.log(res)
    })
})

true

[ 'london', 'paris' ]


### SET
SET in Redis is list with only unique values (unordered). Basic operations are `SADD`, `SREM`, `SISMEMBER` (check if value present in SET), `SMEMBERS` (return all members of the SET).

In [14]:
client.sadd('colors', 'red', 'blue', 'cyan', 'red', 'yellow')
client.smembers('colors', print)

true

[ 'yellow', 'blue', 'cyan', 'red' ]


In [15]:
client.sismember('colors', 'blue', print)

true

1


In [16]:
// Get number of items in set
client.scard('colors', print)

true

4


In [17]:
// Removing specific and random item from SET
client.srem('colors', 'blue')
client.spop('colors') // removes random item from SET
client.smembers('colors', print)

true

[ 'cyan', 'red' ]


Redis SETS have functionality to get union, intersection, difference using the `SUNION`, `SINTER`, `SDIFF`

In [23]:
client.sadd('set1', 'a', 'b', 'c', 'd');
client.sadd('set2', 'c', 'd', 'e', 'f');

// Intersection
client.sinter('set1', 'set2', print)
// Save intersection in a separate key
client.sinterstore('intersection', 'set1', 'set2')
client.smembers('intersection', print)

// Union
client.sunionstore('union', 'set1', 'set2')
client.smembers('union', print)

// Difference (items in set a but not in set b, order matters)
client.sdiffstore('difference', 'set1', 'set2')
client.smembers('difference', print)

true

[ 'c', 'd' ]
[ 'c', 'd' ]
[ 'f', 'e', 'c', 'a', 'b', 'd' ]
[ 'b', 'a' ]


### HASH
Redis HASH data type lets us store group of key value pairs. Both the key and the value are strings. In a lot of ways HASH can be considered as miniature version of Redis itself. Operations: `HSET`, `HGET`, `HGETALL`, `HDEL`, etc.

In [24]:
client.hset('capitals', 'india', 'new delhi')
client.hget('capitals', 'india', print)

true

new delhi


In [25]:
// We set one key value pair at a time
client.hset('capitals', 'france', 'paris')
client.hset('capitals', 'norway', 'oslo')
client.hset('capitals', 'australia', 'canberra')
client.hgetall('capitals', print)

true

{ india: 'new delhi',
  france: 'paris',
  norway: 'oslo',
  australia: 'canberra' }


In [32]:
// Or many at a time (HMSET deprecated as per Redis 4.0)
client.hmset('cars', {
    '911': 'Porsche',
    'GTR': 'Nissan',
    'SLS': 'Mercedes',
    'M5': 'BMW',
    'Evoque': 'Land Rover'
})

// Get list of keys
client.hmget('cars', ['911', 'Evoque'], print)

true

[ 'Porsche', 'Land Rover' ]


In [26]:
// Count number of keys
client.hlen('capitals', print)

true

4


In [27]:
// Check if a key exists
client.hexists('capitals', 'bangladesh', print)

true

0


In [30]:
// Get all keys
client.hkeys('capitals', print)
// Get all values
client.hvals('capitals', print)

true

[ 'india', 'france', 'norway', 'australia' ]
[ 'new delhi', 'paris', 'oslo', 'canberra' ]


Often times we store numeric values in redis HASH, so Redis provides helper commands to increment/ decrement those.

In [33]:
client.hset('nums', 'one', '1')
client.hincrby('nums', 'one', -5)
client.hget('nums', 'one', print)

true

-4


### ZSET
ZSET is sorted SET. We need to provide a numeric score for each value. The score determines how the items will be sorted

In [53]:
client.zadd('pokemons', 714, 'blissey')
client.zadd('pokemons', 544, 'wailord')
client.zadd('pokemons', 344, 'scizor')
client.zadd('pokemons', 354, 'sunflora')
client.zadd('pokemons', 360, 'charizard')
client.zrem('pokemons', 'blissey')
// Get count of elements added
client.zcard('pokemons', print)

true

4


In [49]:
// Get count of all pokemons having HP between 300 and 600
client.zcount('pokemons', 300, 600, print)

true

4


In [51]:
// Get rank of a pokemon
client.zscore('pokemons', 'sunflora', print)

true

354


In [52]:
client.zrangebyscore('pokemons', 300, 500, print)

true

[ 'scizor', 'sunflora', 'charizard' ]
